In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Libraries
- Seaborn
- Matplotlib
- train_test_split 
- Random Forest Classifier (sklearn)
- accuracy_score
- confusion_matrix
- classification_report

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Reading data

In [ ]:
df_train=pd.read_csv("/kaggle/input/glass-identification-database/glassm-data.csv")
df_train.head()

### Information of train data
- As we can see Column "Ca" has digits but data type is Object.

In [ ]:
df_train.info()

### Column conversion
- Convert Column "Ca" to float data type
- errors="coerce" : If ‘coerce’, then invalid parsing will be set as NaN.

In [ ]:
df_train["Ca"]=pd.to_numeric(df_train["Ca"],errors="coerce")

In [ ]:
df_train["Ca"].dtypes

### Drop Column
- Id column

In [ ]:
df_train=df_train.drop(["Id"],axis=1)

###  Descriptive statistics
- Count
- Mean
- Standard deviation
- Minimum
- 25% ,50%,75%
- Maximum

In [ ]:
df_train.describe()

### Null values check

In [ ]:
df_train.isnull().sum()

### Replacing null vlaues
 - Replacing null values with mean of that column

In [ ]:
for column in df_train.columns:
    df_train[column] = df_train[column].fillna(df_train[column].mean())

### Check Null values again
- No null values present

In [ ]:
df_train.isnull().sum()

### Checking outliers and replacing them by upper limit and lower limit of that particular column

In [ ]:
def lowerupper(col):
    q1=np.quantile(col,.25)
    q3=np.quantile(col,.75)
    inter=q3-q1
    lowerbound=q1-(inter*3.0)
    upperbound=q3+(inter*3.0)
    outlier=[]
    for x in col:
        if((x>upperbound)or(x<lowerbound)):
            outlier.append(x)
    if outlier==[]:
        print("No outlier present")
    else:
        return outlier

In [ ]:
ri_ot=lowerupper(df_train["RI"])
print("Outlier Count in column RI:",ri_ot)

na_ot=lowerupper(df_train["Na"])
print("Outlier Count in column Na:",na_ot)

mg_ot=lowerupper(df_train["Mg"])
print("Outlier Count in column Mg:",mg_ot)

al_ot=lowerupper(df_train["Al"])
print("Outlier Count in column Al:",al_ot)

si_ot=lowerupper(df_train["Si"])
print("Outlier Count in column Si:",si_ot)

ka_ot=lowerupper(df_train["Ka"])
print("Outlier Count in column Ka:",ka_ot)

ca_ot=lowerupper(df_train["Ca"])
print("Outlier Count in column Ca:",ca_ot)

ba_ot=lowerupper(df_train["Ba"])
print("Outlier Count in column Ba:",ba_ot)

fe_ot=lowerupper(df_train["Fe"])
print("Outlier Count in column Fe:",fe_ot)

In [ ]:
def loweruppervalue(col):
    q1=np.quantile(col,.25)
    q3=np.quantile(col,.75)
    inter=q3-q1
    lowerbound=q1-(inter*3.0)
    upperbound=q3+(inter*3.0)
    return lowerbound,upperbound

In [ ]:
rilp,riup=loweruppervalue(df_train["RI"])
print("Lowerbound: ",rilp,"Upperbound :",riup)
df_train["RI"] = np.where(df_train["RI"] < rilp,rilp, df_train["RI"])
df_train["RI"] = np.where(df_train["RI"] > riup,riup ,df_train["RI"])

nalp,naup=loweruppervalue(df_train["Na"])
print("Lowerbound: ",nalp,"Upperbound :",naup)
df_train["Na"] = np.where(df_train["Na"] < nalp,nalp, df_train["Na"])
df_train["Na"] = np.where(df_train["Na"] > naup,naup ,df_train["Na"])

allp,alup=loweruppervalue(df_train["Al"])
print("Lowerbound: ",allp,"Upperbound :",alup)
df_train["Al"] = np.where(df_train["Al"] < allp,allp, df_train["Al"])
df_train["Al"] = np.where(df_train["Al"] > alup,alup ,df_train["Al"])

silp,siup=loweruppervalue(df_train["Si"])
print("Lowerbound: ",silp,"Upperbound :",siup)
df_train["Si"] = np.where(df_train["Si"] < silp,silp, df_train["Si"])
df_train["Si"] = np.where(df_train["Si"] > siup,siup ,df_train["Si"])

kalp,kaup=loweruppervalue(df_train["Ka"])
print("Lowerbound: ",kalp,"Upperbound :",kaup)
df_train["Ka"] = np.where(df_train["Ka"] < kalp,kalp, df_train["Ka"])
df_train["Ka"] = np.where(df_train["Ka"] > kaup,kaup ,df_train["Ka"])

calp,caup=loweruppervalue(df_train["Ca"])
print("Lowerbound: ",calp,"Upperbound :",caup)
df_train["Ca"] = np.where(df_train["Ca"] < calp,calp, df_train["Ca"])
df_train["Ca"] = np.where(df_train["Ca"] > caup,caup ,df_train["Ca"])

balp,baup=loweruppervalue(df_train["Ba"])
print("Lowerbound: ",balp,"Upperbound :",baup)
df_train["Ba"] = np.where(df_train["Ba"] < balp,balp, df_train["Ba"])
df_train["Ba"] = np.where(df_train["Ba"] > balp,balp ,df_train["Ba"])

felp,feup=loweruppervalue(df_train["Fe"])
print("Lowerbound: ",felp,"Upperbound :",feup)
df_train["Fe"] = np.where(df_train["Fe"] < felp,felp, df_train["Fe"])
df_train["Fe"] = np.where(df_train["Fe"] > feup,feup ,df_train["Fe"])

In [ ]:
ri_ot=lowerupper(df_train["RI"])
print("Outlier Count in column RI:",ri_ot)

na_ot=lowerupper(df_train["Na"])
print("Outlier Count in column Na:",na_ot)

mg_ot=lowerupper(df_train["Mg"])
print("Outlier Count in column Mg:",mg_ot)

al_ot=lowerupper(df_train["Al"])
print("Outlier Count in column Al:",al_ot)

si_ot=lowerupper(df_train["Si"])
print("Outlier Count in column Si:",si_ot)

ka_ot=lowerupper(df_train["Ka"])
print("Outlier Count in column Ka:",ka_ot)

ca_ot=lowerupper(df_train["Ca"])
print("Outlier Count in column Ca:",ca_ot)

ba_ot=lowerupper(df_train["Ba"])
print("Outlier Count in column Ba:",ba_ot)

fe_ot=lowerupper(df_train["Fe"])
print("Outlier Count in column Fe:",fe_ot)

### Histogram

In [ ]:
plt.subplot(1,2,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("RI")
plt.hist(df_train["RI"])

plt.subplot(1,2,2)
plt.title("Na")
plt.hist(df_train["Na"])


In [ ]:
plt.subplot(1,2,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("Mg")
plt.hist(df_train["Mg"])

plt.subplot(1,2,2)
plt.title("Al")
plt.hist(df_train["Al"])


In [ ]:
plt.subplot(1,2,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("Si")
plt.hist(df_train["Si"])

plt.subplot(1,2,2)
plt.title("Ka")
plt.hist(df_train["Ka"])


In [ ]:
plt.subplot(1,2,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("Ca")
plt.hist(df_train["Ca"])

plt.subplot(1,2,2)
plt.title("Ba")
plt.hist(df_train["Ba"])


In [ ]:
plt.subplot(1,2,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("Fe")
plt.hist(df_train["Fe"])

plt.subplot(1,2,2)
plt.title("Type")
plt.hist(df_train["Type"])


### Barplot

In [ ]:
plt.subplot(1,3,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("RI")
sb.barplot(data=df_train,x="Type",y="RI")

plt.subplot(1,3,2)
plt.title("Na")
sb.barplot(data=df_train,x="Type",y="Na")

plt.subplot(1,3,3)
plt.title("Mg")
sb.barplot(data=df_train,x="Type",y="Mg")


In [ ]:
plt.subplot(1,3,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("RI")
sb.barplot(data=df_train,x="Type",y="Al")

plt.subplot(1,3,2)
plt.title("Na")
sb.barplot(data=df_train,x="Type",y="Si")

plt.subplot(1,3,3)
plt.title("Mg")
sb.barplot(data=df_train,x="Type",y="Ka")


In [ ]:
plt.subplot(1,3,1)
plt.subplots_adjust(left=0,right=2,bottom=1,top=2,wspace=0.2,hspace=0.4)
plt.title("Ca")
sb.barplot(data=df_train,x="Type",y="Ca")

plt.subplot(1,3,2)
plt.title("Ba")
sb.barplot(data=df_train,x="Type",y="Ba")

plt.subplot(1,3,3)
plt.title("Fe")
sb.barplot(data=df_train,x="Type",y="Fe")


### Pie Chart
-  Column "Type"

In [ ]:
llables=[1,2,3,5,6,7]
plt.figure()
plt.pie(df_train.groupby(["Type"]).size(),labels=llables)
plt.legend()
plt.show()

### Splitting into x and y

In [ ]:
X=df_train[['RI', 'Na', 'Mg', 'Al', 'Si', 'Ka', 'Ca', 'Ba', 'Fe']]
y=df_train['Type']

In [ ]:
X.shape,y.shape

### Oversampling of data

In [ ]:
os=RandomOverSampler()
X_train_new,y_train_new=os.fit_resample(X, y)

In [ ]:
X_train_new.shape,y_train_new.shape

### Splitting data again
- train and test

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_train_new,y_train_new,test_size=0.30,random_state=707)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
y_train.shape,y_test.shape

### RandomForestClassifier

In [ ]:
model=RandomForestClassifier().fit(X_train,y_train)
pred_ram=model.predict(X_test)

### Accuracy score

In [ ]:
print("The accuracy score is :",accuracy_score(y_test,pred_ram))

### Confusion Matrix

In [ ]:
confusion_matrix(y_test, pred_ram)

### Classification Report

In [ ]:
print(classification_report(y_test, pred_ram))